In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # done
from google.colab import files
uploaded = files.upload()

# The file was likely uploaded as 'Food_Delivery_Times (1).csv'. Change the filename here to match
df = pd.read_csv('Food_Delivery_Times (1).csv')
df.head()

Saving Food_Delivery_Times.csv to Food_Delivery_Times (1).csv


,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,738,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,741,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,661,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,412,19.03,Clear,Low,Morning,Bike,16,5.0,68


In [ ]:
df.info() #Done

OOP Proof of Concept

In [ ]:
class Deliverer:
    def __init__(self, vehicle_type):
        self.vehicle_type = vehicle_type
        self.orders = []

    def add_distance(self, distance_km):
        self.orders.append(distance_km)

    def average_distance(self):
        if not self.orders:
            return 0
        return sum(self.orders) / len(self.orders)

# Create dictionary to store Deliverer objects by vehicle type
deliverers = {}

# Loop through dataframe rows to populate the dictionary
for _, row in df.iterrows():
    vehicle_type = row['Vehicle_Type']
    distance = row['Distance_km']

    if vehicle_type not in deliverers:
        deliverers[vehicle_type] = Deliverer(vehicle_type)

    deliverers[vehicle_type].add_distance(distance)

# Output mean distance by vehicle type
print("Average Delivery Distance by Vehicle Type:")
for vehicle, person in deliverers.items():
    print(f"{vehicle.capitalize()}: {person.average_distance():.2f} km")

Average Delivery Distance by Vehicle Type:
Scooter: 10.00 km
Bike: 9.99 km
Car: 10.34 km


How does distance affect order frequency?

In [ ]:
sns.histplot(data = df,
             x = 'Distance_km',
             binwidth = 5) #done

How does weather affect delivery time?

In [ ]:
df.groupby('Weather')['Delivery_Time_min'].mean() #done

What is the relationship between vehicle type and delivery time?

In [ ]:
bins = range(0, int(df['Distance_km'].max()))[0 : int(df['Distance_km'].max()) : 5]
df.groupby(['Vehicle_Type', pd.cut(df.Distance_km, bins)])['Delivery_Time_min'].mean() # done

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Weather', hue='Traffic_Level')
plt.title('Weather vs Traffic Level')
plt.ylabel('Count')
plt.xlabel('Weather Condition')
plt.legend(title='Traffic Level')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
df.groupby('Vehicle_Type')['Distance_km'].mean() #done

How does order distance affect delivery vehicle?

In [ ]:
sns.histplot(data = df,
             hue = 'Vehicle_Type',
             x = 'Distance_km',
             binwidth = 5)
plt.title('Order distance vs Vehicle Type')

What is the relationship between weather and traffic level?

In [ ]:
df[['Weather', 'Traffic_Level']].value_counts(sort = False) #Traffic

Does Courier experience affect delivery times, assuming similar traffic levels?

In [ ]:
df.groupby(['Courier_Experience_yrs', 'Traffic_Level'])['Delivery_Time_min'].mean() #traffic

In [ ]:
df_dummies = pd.get_dummies(df_selected, dtype=float)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

targets = df['Delivery_Time_min']
features = pd.get_dummies(df, dtype = float).drop(['Delivery_Time_min', 'Order_ID'], axis = 1).fillna(value = 0)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.2)

lasso = Lasso()
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

lasso.fit(X_train, y_train)

y_pred_lasso = lasso.predict(X_test)

In [ ]:
r2_score(y_test, y_pred_lasso)

In [ ]:
lr_df = {'Factors' : features.columns, 'Weights' : lasso.coef_}
sns.barplot(data = lr_df,
            x = 'Factors',
            y = 'Weights')
plt.xticks(rotation = 90)
plt.title('Relative Importance of Numeric Features')